### Téléchargement des datas


In [ ]:
import gdown
from os.path import exists

#Download the files only if they are not already in the environnement d'exécution

files_already_downloaded = exists('topics_linked.csv')

if not files_already_downloaded :

  #Import interventions.csv
  url = 'https://drive.google.com/uc?id=1e8J6m_HV66F_yHqjvzscePSLtvZi6YPB'
  output = 'interventions.csv'
  gdown.download(url, output, quiet=False)

  #Import Global_Mobility_Report.csv
  url = 'https://drive.google.com/uc?id=1GyWeePY8xG3G8DRlI0utcTsCmNCYjo3L'
  output = 'Global_Mobility_Report.csv'
  gdown.download(url, output, quiet=False)

  #Import Global_Mobility_Report.csv
  url = 'https://drive.google.com/uc?id=1GyWeePY8xG3G8DRlI0utcTsCmNCYjo3L'
  output = 'Global_Mobility_Report.csv'
  gdown.download(url, output, quiet=False)

  #Import apple mobility trends
  url = 'https://drive.google.com/uc?id=1Mg-1rxmnnPQIdWKsPWN9apH-y3P2x9dK'
  output = 'applemobilitytrends-2020-04-20.csv'
  gdown.download(url, output, quiet=False)

  #Import aggregated time series
  url = 'https://drive.google.com/uc?id=1lTjzeqc3wDQhcxG4VC9qtbw06vHED2eX'
  output = 'aggregated_timeseries.json'
  gdown.download(url, output, quiet=False)

  #Import topics_linked
  url = 'https://drive.google.com/uc?id=1Pv1EKCNR72Yxivrvn_KDj7vfr2vKEmfh'
  output = 'topics_linked.csv'
  gdown.download(url, output, quiet=False)

else :
  print ('Files already downloaded')

Downloading...
From: https://drive.google.com/uc?id=1e8J6m_HV66F_yHqjvzscePSLtvZi6YPB
To: /content/interventions.csv
100%|██████████| 1.01k/1.01k [00:00<00:00, 1.21MB/s]
Downloading...
From: https://drive.google.com/uc?id=1GyWeePY8xG3G8DRlI0utcTsCmNCYjo3L
To: /content/Global_Mobility_Report.csv
100%|██████████| 149M/149M [00:01<00:00, 88.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1GyWeePY8xG3G8DRlI0utcTsCmNCYjo3L
To: /content/Global_Mobility_Report.csv
100%|██████████| 149M/149M [00:00<00:00, 202MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Mg-1rxmnnPQIdWKsPWN9apH-y3P2x9dK
To: /content/applemobilitytrends-2020-04-20.csv
100%|██████████| 258k/258k [00:00<00:00, 33.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1lTjzeqc3wDQhcxG4VC9qtbw06vHED2eX
To: /content/aggregated_timeseries.json
100%|██████████| 129M/129M [00:01<00:00, 78.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Pv1EKCNR72Yxivrvn_KDj7vfr2vKEmfh
To: /content/topics_linked.cs

### Visualisation des différents fichiers

In [ ]:
import pandas as pd
import json

#Load csv files
df_interventions = pd.read_csv('interventions.csv')
df_google_mobility = pd.read_csv('Global_Mobility_Report.csv')
df_apple_mobility = pd.read_csv('applemobilitytrends-2020-04-20.csv')
df_topics_linked = pd.read_csv('topics_linked.csv')

#Load json file
f = open('aggregated_timeseries.json')
json_timeseries = json.load(f)
f.close()


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
#Print the columns of the dataframes
print (df_interventions.columns)
print (df_google_mobility.columns)
print (df_apple_mobility.columns)
print (df_topics_linked.columns)

#Print the keys of the json file
print (json_timeseries.keys())